In [5]:
from keras.callbacks import ModelCheckpoint, TensorBoard


from Models import FCN8, FCN32, SegNet, UNet
from keras import optimizers
import math

#############################################################################

train_batch_size = 4
n_classes = 2

epochs = 5

input_height = 256
input_width = 256




key = "segnet"


##################################

method = {
    "fcn32": FCN32.FCN32,
    "fcn8": FCN8.FCN8,
    'segnet': SegNet.SegNet,
    'unet': UNet.UNet}

m = method[key](n_classes, input_height=input_height, input_width=input_width)
m.compile(
    loss='categorical_crossentropy',
    optimizer="adadelta",
    metrics=['acc'])
###################################################

def generateData(batch_size=4,img_w=256,img_h=256,n_label=2,image_names=['test.tif'],label_names=['test_label.png']): 
    print ('gen-Sub-Image-Data...')
    image_filepath ='D:\Python\seg-data\data_MB/'
    batch_num=0
    while True:   
        bs=batch_size
        
        dataset = gdal.Open(image_filepath+image_names[batch_num%len(image_names)])
        im_width = dataset.RasterXSize #栅格矩阵的列数
        im_height = dataset.RasterYSize #栅格矩阵的行数
        image_data = dataset.ReadAsArray(0,0,im_width,im_height)
        label_data=cv2.imread(image_filepath+label_names[batch_num%len(image_names)],cv2.IMREAD_GRAYSCALE)
        train_data = []  
        train_label =  []  
        for i in range(bs):
            random_width = random.randint(0, im_width - img_w - 1)
            random_height = random.randint(0, im_height - img_h - 1)
            bands_roi=[]
            for band in image_data :
                band_roi = band[random_height: random_height + img_h, random_width: random_width + img_w]
                bands_roi.append(band_roi)
            data_roi=bands_roi
            #to_categorical(train_label, num_classes=n_label)  
            label_roi = to_categorical((label_data[random_height: random_height + img_h, random_width: random_width + img_w]).flatten(), num_classes=n_label)
            train_data.append( data_roi)  
            train_label.append(label_roi)
        #yield(np.array(train_data).shape,np.array(train_label).shape)    
        yield(np.array(train_data),np.array(train_label))
        batch_num=batch_num+1
#image_names_set=['test.tif']
#label_names_set=['test_label.png']
#for i in(generateData(8,256,256,2,image_names_set,label_names_set)):
#    print(i)#########################
checkpoint = ModelCheckpoint(
    filepath="output/%s_model.h5" %
    key,
    monitor='acc',
    mode='auto',
    save_best_only='True')
tensorboard = TensorBoard(log_dir='output/log_%s_model' % key)

m.fit_generator(generator=generateData(2,256,256,2,['test.tif'],['test_label.png']),
                            steps_per_epoch=10,
                            epochs=10,
                            verbose=1,
                            validation_data=generateData(BS,img_w,img_h,2,['test.tif'],['test_label.png']),
                            validation_steps=2,
                            callbacks=callable,
                            max_q_size=1)  


NameError: name 'train_numb' is not defined